In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import lxml.html as lh

In [2]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

url = "https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1"
years = ['2019','2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005']
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
def build_by_year(year):
    page = 'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/plus/?saison_id=' + year
    tree = requests.get(page, headers = headers)
    soup = BeautifulSoup(tree.content, 'html.parser')
    spending_table = soup.select("tbody")[1]
    names = soup.select("td.hide-for-pad > a.vereinprofil_tooltip")
    season = []
    for name in names:
        temp = []
        temp_name = name.text.rstrip('FC').lower().rstrip()
        temp.append(year)
        temp.append(temp_name)
        temp.append(name.get('href'))
        season.append(temp)
    return season

In [4]:
years_urls = []

for year in years:
    built_year = build_by_year(year)
    years_urls.extend(built_year)

In [5]:
years_urls

[['2019',
  'manchester city',
  '/manchester-city/startseite/verein/281/saison_id/2019'],
 ['2019', 'liverpool', '/fc-liverpool/startseite/verein/31/saison_id/2019'],
 ['2019',
  'tottenham hotspur',
  '/tottenham-hotspur/startseite/verein/148/saison_id/2019'],
 ['2019', 'chelsea', '/fc-chelsea/startseite/verein/631/saison_id/2019'],
 ['2019',
  'manchester united',
  '/manchester-united/startseite/verein/985/saison_id/2019'],
 ['2019', 'arsenal', '/fc-arsenal/startseite/verein/11/saison_id/2019'],
 ['2019', 'everton', '/fc-everton/startseite/verein/29/saison_id/2019'],
 ['2019',
  'leicester city',
  '/leicester-city/startseite/verein/1003/saison_id/2019'],
 ['2019',
  'west ham united',
  '/west-ham-united/startseite/verein/379/saison_id/2019'],
 ['2019',
  'afc bournemouth',
  '/afc-bournemouth/startseite/verein/989/saison_id/2019'],
 ['2019',
  'wolverhampton wanderers',
  '/wolverhampton-wanderers/startseite/verein/543/saison_id/2019'],
 ['2019',
  'southampton',
  '/fc-southampt

In [12]:
df_years_urls = pd.DataFrame(years_urls)
df_years_urls.columns = ['Year', 'Team', 'URL']
df_years_urls

,Year,Team,URL
0,2019,manchester city,/manchester-city/startseite/verein/281/saison_...
1,2019,liverpool,/fc-liverpool/startseite/verein/31/saison_id/2019
2,2019,tottenham hotspur,/tottenham-hotspur/startseite/verein/148/saiso...
3,2019,chelsea,/fc-chelsea/startseite/verein/631/saison_id/2019
4,2019,manchester united,/manchester-united/startseite/verein/985/saiso...
5,2019,arsenal,/fc-arsenal/startseite/verein/11/saison_id/2019
6,2019,everton,/fc-everton/startseite/verein/29/saison_id/2019
7,2019,leicester city,/leicester-city/startseite/verein/1003/saison_...
8,2019,west ham united,/west-ham-united/startseite/verein/379/saison_...
9,2019,afc bournemouth,/afc-bournemouth/startseite/verein/989/saison_...


In [42]:
# returns 2 dicts, 1 of spent, 1 of sales
def get_all_transfer_info_year(url, year):
    date_dict = {
    '19/20': '2019',
    '18/19': '2018',
    '17/18': '2017',
    '16/17': '2016',
    '15/16': '2015',
    '14/15': '2014',
    '13/14': '2013',
    '12/13': '2012',
    '11/12': '2011',
    '10/11': '2010',
    '09/10': '2009',
    '08/09': '2008',
    '07/08': '2007',
    '06/07': '2006',
    '05/06': '2005',
    '04/05': '2004'}
    year_to_ret = ''
    for dash, whole in date_dict.items():
        if year == whole:
            year_to_ret = dash
    fixed_url = url.replace("startseite", "alletransfers")[1:]
    full_url = 'https://www.transfermarkt.com/' + fixed_url 
    tree = requests.get(full_url, headers = headers)
    s = BeautifulSoup(tree.content, 'html.parser')
    incoming = s.select("td.redtext")
    in_and_out = []
    for idx, spent in enumerate(incoming):
        for parent in incoming[idx].parents:
            if parent.get('class') == ['box']:
                split = parent.text.split()
                if split[0] == 'Arrivals' and split[1] == year_to_ret:
                    in_and_out = [spent.text, '']
                    break
            
        # if we got here, we didnt find it!
    if not bool(in_and_out):
        in_and_out = ['0', '']
    outgoing = s.select("td.greentext")    
    for idx, sale in enumerate(outgoing):
        for parent in outgoing[idx].parents:
            if parent.get('class') == ['box']:
                split = parent.text.split()
                if split[0] == 'Departures' and split[1] == year_to_ret:
                    in_and_out[1] = sale.text
    
    if in_and_out[1] == '':
        in_and_out[1] =  '0'
    return in_and_out

In [ ]:
team_info_list = []

for i in range(len(df_years_urls)): 
    results = get_all_transfer_info_year(df_years_urls.loc[i, "URL"], df_years_urls.loc[i, "Year"])